In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ENTRENAMIENTO DEL MODELO (CNN SIMPLE)

### 1. Transformaciones

In [3]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

train_data = datasets.ImageFolder('data/train', transform=transform)
val_data   = datasets.ImageFolder('data/val', transform=transform)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=16)

#### 2. Modelo CNN simple

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*16*16, 64),
            nn.ReLU(),
            nn.Linear(64, 2)     
        )
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008)

#### 3. Entrenamiento

In [5]:
for epoch in range(8):
    model.train()
    total_loss = 0

    for X,y in train_loader:
        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred,y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, loss = {total_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), "modelo_gatos_perros.pth")

Epoch 1, loss = 0.6343
Epoch 2, loss = 0.5285
Epoch 3, loss = 0.4510
Epoch 4, loss = 0.3770
Epoch 5, loss = 0.2899
Epoch 6, loss = 0.1934
Epoch 7, loss = 0.1056
Epoch 8, loss = 0.0463
